In [11]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', 1000)
# pd.set_option('display.max_rows', 100)

In [12]:
import sqlite3
import pandas as pd
import os
from pathlib import Path


def create_connection(db_path):
    """Cria a conexão com o banco de dados SQLite."""
    connection = None
    try:
        connection = sqlite3.connect(db_path)

        print("Conexão SQLite estabelecida.")
    except sqlite3.Error as e:
        print(f"Erro ao conectar ao SQLite: {e}")
    return connection

BASE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
DATA_DIR = os.path.join(BASE_DIR, 'database')
FT_DIR = os.path.join(BASE_DIR, 'modelagem', 'feature_eng', 'data')

conn = create_connection(db_path=Path(os.path.join(DATA_DIR, "soccer_data.db")))
country = "Brazil"
query = "SELECT * FROM soccer_data WHERE country = ?"

df_hist_soccer_matches = pd.read_sql_query(query, conn, params=(country,))

conn.close()


Conexão SQLite estabelecida.


In [13]:
df_hist_soccer_matches.head()

,id,country,league,season,home_team,away_team,home_score,away_score,result,psch,pscd,psca,maxch,maxcd,maxca,avgch,avgcd,avgca,bfech,bfecd,datetime,hash,last_updated
0,5415,Brazil,Serie A,2012,Palmeiras,Portuguesa,1,1,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90,NaN,NaN,2012-05-19 22:30:00,10444097902145517897,2024-12-18 22:32:46
1,5416,Brazil,Serie A,2012,Sport Recife,Flamengo RJ,1,1,D,2.83,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58,NaN,NaN,2012-05-19 22:30:00,7876314183501917566,2024-12-18 22:32:46
2,5417,Brazil,Serie A,2012,Figueirense,Nautico,2,1,H,1.60,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64,NaN,NaN,2012-05-20 01:00:00,9296066046964045682,2024-12-18 22:32:46
3,5418,Brazil,Serie A,2012,Botafogo RJ,Sao Paulo,4,2,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84,NaN,NaN,2012-05-20 20:00:00,3618841616446699339,2024-12-18 22:32:46
4,5419,Brazil,Serie A,2012,Corinthians,Fluminense,0,1,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89,NaN,NaN,2012-05-20 20:00:00,11994628649421207242,2024-12-18 22:32:46


## Agrupando dados 

In [14]:
# def group_infos(df_home_team, away_team):
#     df_home_team = df_home_team.copy()
#     away_team = away_team.copy()
#     group_result_home_team = df_home_team.groupby(["home_team"]).agg(
#         gols_marcado_casa = ("home_score", "sum"),
#         gols_sofridos_casa = ("away_score", "sum"),
#         jogos_em_casa = ("away_score", "count"),
#     ).reset_index(names="time")

#     group_result_away_score = away_team.groupby(["away_team"]).agg(
#         gols_marcado_fora = ("away_score", "sum"),
#         gols_sofridos_fora = ("home_score", "sum"),
#         jogos_em_fora = ("away_score", "count"),
#     ).reset_index(names="time")

#     df_merge_data = pd.merge(group_result_home_team, group_result_away_score)
#     return df_merge_data



## Calcumando a força de Ataque e Defesa

In [15]:
import numpy as np
import scipy.stats as stats

# Agrupando dados
def get_group_infos(away_team):
    group_result_home_team = away_team.groupby(["home_team"]).agg(
        gols_marcado_casa = ("home_score", "sum"),
        gols_sofridos_casa = ("away_score", "sum"),
        jogos_em_casa = ("away_score", "count"),
    ).reset_index(names="time")

    group_result_away_score = away_team.groupby(["away_team"]).agg(
        gols_marcado_fora = ("away_score", "sum"),
        gols_sofridos_fora = ("home_score", "sum"),
        jogos_em_fora = ("away_score", "count"),
    ).reset_index(names="time")

    df_merge_data = pd.merge(group_result_home_team, group_result_away_score)
    return df_merge_data

def calc_attack_and_defense_force(df_merge_data):
    df = df_merge_data.copy()
    # Gols marcados pelo time A em casa / Jogos em casa 
    df["attack_force_home_team"] = df["gols_marcado_casa"]/df["jogos_em_casa"].replace(0, 1)
    # Gols sofridos pelo time A em casa / Jogos em casa 
    df["defense_force_home_team"] = df["gols_sofridos_casa"]/df["jogos_em_casa"].replace(0, 1)

    # Gols marcados pelo time B fora de casa / Jogos fora de casa 
    df["attack_force_away_team"] = df["gols_marcado_fora"]/df["jogos_em_fora"].replace(0, 1)
    # Gols sofridos pelo time B fora de casa / Jogos fora de casa 
    df["defense_force_away_team"] = df["gols_sofridos_fora"]/df["jogos_em_fora"].replace(0, 1)


    # Normalizando valores
    # Média geral do campeonato: se um time fez muito mais gols que o outro seus numeros podem enganar, por isso da normalização
    league_avg_goals_home = df["gols_marcado_casa"].sum() / df["jogos_em_casa"].sum()
    league_avg_goals_away = df["gols_marcado_fora"].sum() / df["jogos_em_fora"].sum()

    # Normalizando
    df["attack_force_home_team"] /= league_avg_goals_home
    df["defense_force_home_team"] /= league_avg_goals_away
    df["attack_force_away_team"] /= league_avg_goals_away
    df["defense_force_away_team"] /= league_avg_goals_home


    # Gerando estimativas de gols esperados com a normalização
    df["expected_goals_home_team"] = league_avg_goals_home * df["attack_force_home_team"] * df["defense_force_away_team"]
    df["expected_goals_away_team"] = league_avg_goals_away * df["attack_force_away_team"] * df["defense_force_home_team"]

    return df


def get_goal_distribution(lambda_time: float, k_values: int = 6):
    """
    Calcula a distribuição de Poisson para gols esperados de um time.

    :param lambda_time: Média esperada de gols do time (float)
    :param k_values: Número máximo de gols a considerar (int)
    :return: Número de gols mais provável, probabilidade desse número de gols, e um dicionário com todas as probabilidades
    """
    k_range_values = np.arange(k_values)

    # Calcula a probabilidade para cada número de gols
    poisson_prob = stats.poisson.pmf(k_range_values, lambda_time)

    # Encontra o número de gols mais provável
    num_goals = np.argmax(poisson_prob)

    # Retorna também um dicionário com todas as probabilidades
    prob_dict = {k: round(prob, 6) for k, prob in zip(k_range_values, poisson_prob)}

    return num_goals, poisson_prob[num_goals], prob_dict

def get_probabilities_of_win_draw_loss(row:pd.Series, max_goals:int=5)->pd.Series:
    """
    Calcula a probabilidade de vitória, empate e derrota para um jogo específico.
    
    Parâmetros:
    row: Linha do DataFrame contendo 'prob_dict_home' e 'prob_dict_away'.
    max_goals: Número máximo de gols considerados.
    
    Retorna:
    (prob_home_win, prob_draw, prob_away_win)
    """
    
    prob_A = row["prob_dict_home"]
    prob_B = row["prob_dict_away"]

    prob_home_win = sum(
        prob_A[i] * sum(prob_B[j] for j in range(i))
        for i in range(1, max_goals+1)
    )

    prob_draw = sum(
        prob_A[i] * prob_B[i]
        for i in range(max_goals+1)
    )

    prob_away_win = sum(
        prob_B[j] * sum(prob_A[i] for i in range(j))
        for j in range(1, max_goals+1)
    )

    # print(f"Vitória do Time A: {prob_home_win:.4f} ({prob_home_win * 100:.2f}%)")
    # print(f"Empate: {prob_draw:.4f} ({prob_draw * 100:.2f}%)")
    # print(f"Vitória do Time B: {prob_away_win:.4f} ({prob_away_win * 100:.2f}%)")

    return prob_home_win, prob_draw, prob_away_win

def predict(df, name_home_team, name_away_team):
    df = df.copy()
    # Selecionar o Atlético-MG como time da casa
    home_team = df[df["time"] == name_home_team].iloc[0]

    # Selecionar o Sport Recife como time visitante
    away_team = df[df["time"] == name_away_team].iloc[0]

    # Criar um dicionário para simular a estrutura de um jogo
    game = {
        "prob_dict_home": home_team["prob_dict_home"],  # Atlético-MG como time da casa
        "prob_dict_away": away_team["prob_dict_away"],  # Sport Recife como visitante
    }

    # Calcular as probabilidades para o jogo específico
    prob_home_win, prob_draw, prob_away_win = get_probabilities_of_win_draw_loss(game)

    # Exibir os resultados
    # print(f"Probabilidade de Vitória do Atlético-MG: {prob_home_win:.2%}")
    # print(f"Probabilidade de Empate: {prob_draw:.2%}")
    # print(f"Probabilidade de Vitória do Sport Recife: {prob_away_win:.2%}")

    # print(prob_home_win, prob_draw, prob_away_win)
    result_text = ["H", "D", "A"]
    return result_text[np.argmax([prob_home_win, prob_draw, prob_away_win])]

In [16]:
df_hist_soccer_matches.head()
df_hist_soccer_matches["datetime"] = pd.to_datetime(df_hist_soccer_matches["datetime"])

In [17]:
def create_table_num_games(df:pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = df.sort_values("datetime")  # Ordenar pelo tempo

    df_games = pd.concat([
        df[["datetime", "home_team"]].rename(columns={"home_team": "team"}),
        df[["datetime", "away_team"]].rename(columns={"away_team": "team"})
    ])

    df_games["games_played"] = 1  # Cada linha representa um jogo para o time
    df_games["total_games"] = df_games.groupby("team")["games_played"].cumsum()

    return df_games

In [18]:

qtd_min_jogos = 5
# Verifica se os time tem a quantidade minima de dias para calcular a probabilidade

df_simulation = df_hist_soccer_matches.iloc[0:500]

for id, row in df_simulation.iterrows():

    if id >= 100:
        break

    df_table_num_games = create_table_num_games(df_simulation)

    df_result = df_table_num_games[df_table_num_games["datetime"] < row["datetime"]]

    if df_result.empty:
        continue
        print("Dados infuficientes para gerar a probabilidade")

    # elif quanty_matchs_home_team < qtd_min_jogos:
    #     continue
    #     print(f"O time da cada tem apenas {quanty_matchs_home_team} jodos e não será possivel gerar a probabilidade")

    # elif quanty_matchs_home_team < qtd_min_jogos:
    #     continue
    #     print(f"O time visitante tem apenas {quanty_matchs_away_team} jodos e não será possivel gerar a probabilidade")
    
    matchs_home_team = df_result[df_result["team"]==row['home_team']]["total_games"]#.values[-1]
    matchs_away_team = df_result[df_result["team"]==row['away_team']]["total_games"]#.values[-1]

    if matchs_home_team.empty or matchs_away_team.empty:
        continue

    quanty_matchs_home_team = matchs_home_team.values[-1]
    quanty_matchs_away_team = matchs_away_team.values[-1]
    # df_times = 
    # df_merge_data = group_infos(df)
    # calc_attack_and_defense_force(df_merge_data)
    
    list_times = [row['home_team'], row['away_team']]
    
    # Parte 2: gerando grupos de informação
    df_roup_infos = get_group_infos(df_simulation)
    df_merge_data = df_roup_infos[df_roup_infos["time"].isin(list_times)]

    # Parte 3: Calculando força do ateque e defesa
    df_attack_and_defense_force = calc_attack_and_defense_force(df_merge_data)

    # Parte 4: Calculando a distribuição de gols


    df_attack_and_defense_force[['num_goals_expected_home', 'prob_num_goals_home', 'prob_dict_home']] = df_attack_and_defense_force['expected_goals_home_team'].apply(lambda x: pd.Series(get_goal_distribution(x)))
    df_attack_and_defense_force[['num_goals_expected_away', 'prob_num_goals_away', 'prob_dict_away']] = df_attack_and_defense_force['expected_goals_away_team'].apply(lambda x: pd.Series(get_goal_distribution(x)))

    # Parte 5: Prevendo resultados
    pred = predict(df_attack_and_defense_force, name_home_team=row['home_team'], name_away_team=row['home_team'])

    df_simulation.loc[id, "pred"] = pred
    # print(quanty_matchs_home_team, quanty_matchs_away_team)

/tmp/ipykernel_469702/146187993.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simulation.loc[id, "pred"] = pred


In [19]:
df_simulation.dropna(subset="pred", inplace=True)

df_simulation[["season","home_team","away_team","home_score","away_score","result","pred",]]

/tmp/ipykernel_469702/2466195758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simulation.dropna(subset="pred", inplace=True)


,season,home_team,away_team,home_score,away_score,result,pred
10,2012,Atletico GO,Ponte Preta,1,1,D,H
11,2012,Flamengo RJ,Internacional,3,3,D,H
12,2012,Portuguesa,Vasco,0,1,A,H
13,2012,Nautico,Cruzeiro,0,0,D,H
14,2012,Atletico-MG,Corinthians,1,0,H,H
...,...,...,...,...,...,...,...
95,2012,Flamengo RJ,Corinthians,0,3,A,H
96,2012,Sao Paulo,Vasco,0,1,A,H
97,2012,Atletico GO,Figueirense,3,2,H,H
98,2012,Coritiba,Palmeiras,1,1,D,H


---

In [26]:
df_hist_soccer_matches["datetime"] = pd.to_datetime(df_hist_soccer_matches["datetime"])
df_hist_soccer_matches.sort_values("datetime", inplace=True)

In [ ]:
# Time A ganhou do time B 1 vez
# Time B ganhou do time C 1 vez

# Qual a força de cada time?

In [146]:
import pandas as pd
from collections import defaultdict
from collections import defaultdict, Counter

import pandas as pd
from collections import defaultdict, deque

# Dados de exemplo
jogos_treino = [
    {"home_team": "Time A", "away_team": "Time B", "home_score": 2, "away_score": 0},
    {"home_team": "Time B", "away_team": "Time C", "home_score": 2, "away_score": 0},

    {"home_team": "Time A", "away_team": "Time D", "home_score": 2, "away_score": 0},
    {"home_team": "Time D", "away_team": "Time C", "home_score": 2, "away_score": 0},

    {"home_team": "Time E", "away_team": "Time B", "home_score": 2, "away_score": 0},
    {"home_team": "Time E", "away_team": "Time C", "home_score": 2, "away_score": 0},
    {"home_team": "Time E", "away_team": "Time D", "home_score": 2, "away_score": 0},

    {"home_team": "Time G", "away_team": "Time A", "home_score": 2, "away_score": 0},
    {"home_team": "Time G", "away_team": "Time D", "home_score": 2, "away_score": 0},
    {"home_team": "Time G", "away_team": "Time D", "home_score": 2, "away_score": 0},

    
]

jogos_teste = [
    {"home_team": "Time A", "away_team": "Time B", "home_score": 2, "away_score": 0},
    {"home_team": "Time B", "away_team": "Time C", "home_score": 2, "away_score": 0},
    {"home_team": "Time A", "away_team": "Time C", "home_score": 2, "away_score": 0},
    {"home_team": "Time C", "away_team": "Time A", "home_score": 1, "away_score": 0},  # Erro
    {"home_team": "Time B", "away_team": "Time A", "home_score": 1, "away_score": 0},  # Erro

]

from collections import defaultdict, deque
def criar_grafo_pesado():

    """
    Exemplo de uso

    > grafo = criar_grafo()

    > grafo["Time A"]["Time B"]["vitorias"] += 2
    > grafo["Time A"]["Time B"]["num_gols"] += 4

    > grafo["Time B"]["Time C"]["vitorias"] += 1
    > grafo["Time B"]["Time C"]["num_gols"] += 3
    """
    return defaultdict(lambda: defaultdict(lambda: {"vitorias": 0, "num_gols": 0}))

def calcular_forca_times(grafo, peso_indireto=0.5):
    forca = defaultdict(float)

    # Iterar sobre uma cópia das chaves do grafo para evitar erro
    for vencedor in list(grafo):
        for perdedor in grafo[vencedor]:
            forca[vencedor] += 1

            # Vitória indireta
            fila = deque([(perdedor, 1)])
            visitados = set()
            while fila:
                atual, nivel = fila.popleft()
                if atual in visitados or nivel > 3:
                    # print("Continua", atual, nivel)
                    if forca.get(atual, None) is None:
                        forca[atual] = 0
                        print("Atribuindo 0 para", atual)
                    continue
                visitados.add(atual)
                for proximo in grafo[atual]:
                    forca[vencedor] += peso_indireto ** nivel
                    fila.append((proximo, nivel + 1))

    return dict(forca)

# Função para construir o grafo de vitórias diretas

def construir_grafo_vitorias_pesado(jogos):
    grafo = defaultdict(Counter)
    grafo_pesado = criar_grafo_pesado()

    grafo
    for jogo in jogos:
        home, away = jogo["home_team"], jogo["away_team"]
        home_score, away_score = jogo["home_score"], jogo["away_score"]

        if home_score > away_score:
            # grafo[home][away] += 1
            grafo_pesado[home][away]["vitorias"] += 1
            grafo_pesado[home][away]["num_gols"] += home_score
            # grafo_pesado[away][home]["num_gols"] += away_score

        elif away_score > home_score:
            # grafo[away][home] += 1
            grafo_pesado[away][home]["vitorias"] += 1
            grafo_pesado[away][home]["num_gols"] += away_score
            # grafo_pesado[home][away]["num_gols"] += home_score
    
    return grafo_pesado

def construir_grafo_vitorias(jogos):
    grafo = defaultdict(set)
    for jogo in jogos:
        home, away = jogo["home_team"], jogo["away_team"]
        home_score, away_score = jogo["home_score"], jogo["away_score"]

        if home_score > away_score:
            grafo[home].add(away)
        elif away_score > home_score:
            grafo[away].add(home)
    return grafo

# Função para verificar se há um caminho (vitória indireta) entre dois times
def existe_caminho(grafo, vencedor, perdedor):
    visitados = set()
    fila = deque([vencedor])

    while fila:
        atual = fila.popleft()
        if atual == perdedor:
            return True
        if atual not in visitados:
            visitados.add(atual)
            fila.extend(grafo[atual] - visitados)
    return False

# Função de avaliação das hipóteses
def avaliar_hipoteses(grafo, jogos_teste):

    forca_times = calcular_forca_times(grafo)

    acertos = erros = 0

    for jogo in jogos_teste:
        home, away = jogo["home_team"], jogo["away_team"]
        home_score, away_score = jogo["home_score"], jogo["away_score"]

        if home_score == away_score:
            continue  # Ignora empates
        

        vencedor = home if home_score > away_score else away
        perdedor = away if home_score > away_score else home

    
        if existe_caminho(grafo, vencedor, perdedor):
            acertos += 1
        else:
            erros += 1

        print(f"Vencedor: {vencedor}, Perdedor: {perdedor} | Força {forca_times.get(vencedor, None)} vs {forca_times.get(perdedor, None)}  | acertos:{acertos} erros:{erros}")

    total = acertos + erros
    taxa_acerto = acertos / total * 100 if total > 0 else 0

    print(f"✅ Hipóteses confirmadas: {acertos}")
    print(f"❌ Hipóteses refutadas: {erros}")
    print(f"📊 Total de hipóteses testadas: {total}")
    print(f"🎯 Taxa de acerto: {taxa_acerto:.2f}%")

# Executa a lógica
grafo_modelo = construir_grafo_vitorias(jogos_treino)
grafo_modelo_pesado = construir_grafo_vitorias_pesado(jogos_treino)
avaliar_hipoteses(grafo_modelo, jogos_teste)


Atribuindo 0 para Time C
Vencedor: Time A, Perdedor: Time B | Força 3.0 vs 1.0  | acertos:1 erros:0
Vencedor: Time B, Perdedor: Time C | Força 1.0 vs 0  | acertos:2 erros:0
Vencedor: Time A, Perdedor: Time C | Força 3.0 vs 0  | acertos:3 erros:0
Vencedor: Time C, Perdedor: Time A | Força 0 vs 3.0  | acertos:3 erros:1
Vencedor: Time B, Perdedor: Time A | Força 1.0 vs 3.0  | acertos:3 erros:2
✅ Hipóteses confirmadas: 3
❌ Hipóteses refutadas: 2
📊 Total de hipóteses testadas: 5
🎯 Taxa de acerto: 60.00%


In [147]:
grafo_modelo_pesado

defaultdict(<function __main__.criar_grafo_pesado.<locals>.<lambda>()>,
            {'Time A': defaultdict(<function __main__.criar_grafo_pesado.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'Time B': {'vitorias': 1, 'num_gols': 2},
                          'Time D': {'vitorias': 1, 'num_gols': 2}}),
             'Time B': defaultdict(<function __main__.criar_grafo_pesado.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'Time C': {'vitorias': 1, 'num_gols': 2}}),
             'Time D': defaultdict(<function __main__.criar_grafo_pesado.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'Time C': {'vitorias': 1, 'num_gols': 2}}),
             'Time E': defaultdict(<function __main__.criar_grafo_pesado.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'Time B': {'vitorias': 1, 'num_gols': 2},
                          'Time C': {'vitorias': 1, 'num_gols': 2},
                          'Time D': {'vitorias': 1, 'num_gols'

defaultdict(<function __main__.criar_grafo.<locals>.<lambda>()>,
            {'Time A': defaultdict(<function __main__.criar_grafo.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'Time B': {'vitorias': 2, 'num_gols': 4}}),
             'Time B': defaultdict(<function __main__.criar_grafo.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'Time C': {'vitorias': 1, 'num_gols': 3}})})

In [136]:
grafo["Time A"]["Time B"]

{'vitorias': 2, 'num_gols': 4}

In [112]:
from pprint import pprint
pprint(grafo_modelo)



defaultdict(<class 'set'>,
            {'Time A': {'Time D', 'Time B'},
             'Time B': {'Time C'},
             'Time C': set(),
             'Time D': {'Time C'},
             'Time E': {'Time C', 'Time D', 'Time B'},
             'Time G': {'Time A', 'Time D'}})


In [ ]:

calcular_forca_times(grafo_modelo)

defaultdict(<class 'set'>, {'Time A': {'Time D', 'Time B'}, 'Time B': {'Time C'}, 'Time D': {'Time C'}, 'Time E': {'Time C', 'Time D', 'Time B'}, 'Time G': {'Time A', 'Time D'}})


RuntimeError: dictionary changed size during iteration

In [113]:
df_season_2023 = df_hist_soccer_matches[df_hist_soccer_matches["season"].isin(['2022', '2023'])]
df_season_2024 = df_hist_soccer_matches[df_hist_soccer_matches["season"] == '2024']

jogos_treino = df_season_2023[["home_team","away_team","home_score","away_score"]].to_dict("records")
jogos_teste = df_season_2024[["home_team","away_team","home_score","away_score"]].iloc[0:-4].to_dict("records")

# Executa a lógica
grafo_modelo = construir_grafo_vitorias(jogos_treino)
avaliar_hipoteses(grafo_modelo, jogos_teste)

Atribuindo 0 para Juventude
Atribuindo 0 para Bahia
Atribuindo 0 para Gremio
Atribuindo 0 para Vasco
Atribuindo 0 para Bragantino
Atribuindo 0 para Athletico-PR
Atribuindo 0 para Atletico GO
Atribuindo 0 para Cuiaba
Atribuindo 0 para America MG
Atribuindo 0 para Flamengo RJ
Atribuindo 0 para Goias
Atribuindo 0 para Santos
Atribuindo 0 para Coritiba
Atribuindo 0 para Cruzeiro
Atribuindo 0 para Sao Paulo
Atribuindo 0 para Corinthians
Atribuindo 0 para Botafogo RJ
Atribuindo 0 para Avai
Atribuindo 0 para Internacional
Atribuindo 0 para Fluminense
Atribuindo 0 para Fortaleza
Atribuindo 0 para Palmeiras
Atribuindo 0 para Atletico-MG
Vencedor: Internacional, Perdedor: Bahia | Força 1868.5 vs 958.625  | acertos:1 erros:0
Vencedor: Fortaleza, Perdedor: Sao Paulo | Força 1475.875 vs 1505.625  | acertos:2 erros:0
Vencedor: Vasco, Perdedor: Gremio | Força 797.125 vs 1383.75  | acertos:3 erros:0
Vencedor: Flamengo RJ, Perdedor: Atletico GO | Força 1711.625 vs 442.0  | acertos:4 erros:0
Vencedor: A

In [83]:
df_season_2024[["home_team","away_team","home_score","away_score"]].iloc[-4:]

,home_team,away_team,home_score,away_score
4930,Bahia,Atletico GO,2,0
4929,Atletico-MG,Athletico-PR,1,0
4932,Bragantino,Criciuma,5,1
4938,Palmeiras,Fluminense,0,1


In [85]:
def estimar_probabilidade_vitoria(grafo, time_a, time_b):
    a_vence_b = existe_caminho(grafo, time_a, time_b)
    b_vence_a = existe_caminho(grafo, time_b, time_a)

    if a_vence_b and not b_vence_a:
        return {time_a: 0.9, time_b: 0.1}
    elif b_vence_a and not a_vence_b:
        return {time_a: 0.1, time_b: 0.9}
    elif a_vence_b and b_vence_a:
        return {time_a: 0.5, time_b: 0.5}
    else:
        # Nenhum caminho encontrado — jogo indefinido
        return {time_a: 0.33, time_b: 0.33, "empate": 0.34}

grafo, time_a, time_b = grafo_modelo, "Corinthians", "Criciuma"
estimar_probabilidade_vitoria(grafo, time_a, time_b)

{'Corinthians': 0.33, 'Criciuma': 0.33, 'empate': 0.34}

In [86]:
existe_caminho(grafo, time_a, time_b)

False

In [ ]:
def verificar_transitividade(grafo, df):
    acertos = 0
    erros = 0
    total = 0

    for time_a in grafo:
        for time_b in grafo[time_a]:
            for time_c in grafo[time_b]:
                if time_c == time_a:  # evitar ciclos triviais
                    continue
                total += 1
                # Verificar se time_a realmente venceu time_c
                confronto = df[
                    ((df["home_team"] == time_a) & (df["away_team"] == time_c)) |
                    ((df["home_team"] == time_c) & (df["away_team"] == time_a))
                ]

                for _, row in confronto.iterrows():
                    if (row["home_team"] == time_a and row["home_score"] > row["away_score"]) or \
                       (row["away_team"] == time_a and row["away_score"] > row["home_score"]):
                        acertos += 1
                    else:
                        erros += 1
    return acertos, erros, total

defaultdict(set, {'Time A': {'Time B', 'Time C'}, 'Time B': {'Time C'}})

In [39]:
import pandas as pd
from collections import defaultdict

def construir_grafo_vitorias(df):
    grafo = defaultdict(set)
    for _, row in df.iterrows():
        home = row["home_team"]
        away = row["away_team"]
        home_score = row["home_score"]
        away_score = row["away_score"]

        if home_score > away_score:
            grafo[home].add(away)
        elif away_score > home_score:
            grafo[away].add(home)
    return grafo

def verificar_transitividade(grafo, df):
    acertos = 0
    erros = 0
    total = 0

    for time_a in grafo:
        for time_b in grafo[time_a]:
            for time_c in grafo[time_b]:
                if time_c == time_a:  # evitar ciclos triviais
                    continue
                total += 1
                # Verificar se time_a realmente venceu time_c
                confronto = df[
                    ((df["home_team"] == time_a) & (df["away_team"] == time_c)) |
                    ((df["home_team"] == time_c) & (df["away_team"] == time_a))
                ]

                for _, row in confronto.iterrows():
                    if (row["home_team"] == time_a and row["home_score"] > row["away_score"]) or \
                       (row["away_team"] == time_a and row["away_score"] > row["home_score"]):
                        acertos += 1
                    else:
                        erros += 1
    return acertos, erros, total

# Filtra temporada
df_season = df_hist_soccer_matches[df_hist_soccer_matches["season"] == '2024']

# Cria grafo de vitórias
grafo = construir_grafo_vitorias(df_season)

# Testa a hipótese de transitividade
acertos, erros, total = verificar_transitividade(grafo, df_season)

print(f"✅ Hipóteses confirmadas: {acertos}")
print(f"❌ Hipóteses refutadas: {erros}")
print(f"📊 Total de hipóteses testadas: {total}")
print(f"🎯 Taxa de acerto: {(acertos / total * 100):.2f}%" if total > 0 else "Sem dados suficientes.")


✅ Hipóteses confirmadas: 1874
❌ Hipóteses refutadas: 2744
📊 Total de hipóteses testadas: 2309
🎯 Taxa de acerto: 81.16%


In [47]:
df_season.shape

(380, 23)

In [32]:
# Disputas historicas entre time A e time B
def get_historical_matches(df:pd.DataFrame, team_a:str, team_b:str) -> pd.DataFrame:
    """
    Filtra os jogos históricos entre dois times específicos.
    
    Parâmetros:
    df: DataFrame contendo os dados dos jogos.
    team_a: Nome do time A (string).
    team_b: Nome do time B (string).
    
    Retorna:
    DataFrame filtrado com os jogos entre os dois times.
    """
    
    # Filtra os jogos onde o time A é o mandante ou o time B é o mandante
    df_filtered = df[((df["home_team"] == team_a) & (df["away_team"] == team_b)) | 
                     ((df["home_team"] == team_b) & (df["away_team"] == team_a))]
    
    return df_filtered

# Exemplo de uso
team_a = "Palmeiras"
team_b = "Botafogo RJ"
historical_matches = get_historical_matches(df_hist_soccer_matches, team_a, team_b)

historical_matches

,id,country,league,season,home_team,away_team,home_score,away_score,result,psch,pscd,psca,maxch,maxcd,maxca,avgch,avgcd,avgca,bfech,bfecd,datetime,hash,last_updated
143,5558,Brazil,Serie A,2012,Botafogo RJ,Palmeiras,1,2,A,1.83,3.63,5.03,1.92,3.63,5.03,1.81,3.37,4.29,NaN,NaN,2012-08-09 01:50:00,8097749072569523101,2024-12-18 22:32:46
334,5749,Brazil,Serie A,2012,Palmeiras,Botafogo RJ,2,2,D,2.04,3.68,3.75,2.04,3.68,4.20,1.92,3.39,3.78,NaN,NaN,2012-11-04 19:00:00,10292364221432528697,2024-12-18 22:32:46
830,6245,Brazil,Serie A,2014,Palmeiras,Botafogo RJ,0,2,A,2.41,3.28,3.27,2.50,3.28,3.60,2.32,3.15,3.05,NaN,NaN,2014-05-28 23:30:00,11594969873000332558,2024-12-18 22:32:46
1020,6435,Brazil,Serie A,2014,Botafogo RJ,Palmeiras,0,1,A,2.50,3.59,2.90,2.50,3.80,3.10,2.42,3.29,2.80,NaN,NaN,2014-10-08 23:30:00,10149767242096788880,2024-12-18 22:32:46
1689,7104,Brazil,Serie A,2016,Botafogo RJ,Palmeiras,3,1,H,2.64,3.52,2.77,2.92,3.54,2.77,2.63,3.26,2.62,NaN,NaN,2016-07-31 22:30:00,1712359202054989202,2024-12-18 22:32:46
1873,7288,Brazil,Serie A,2016,Palmeiras,Botafogo RJ,1,0,H,1.53,4.28,7.28,1.56,4.28,7.50,1.52,3.91,6.48,NaN,NaN,2016-11-20 19:00:00,4653750769588602250,2024-12-18 22:32:46
2073,7488,Brazil,Serie A,2017,Botafogo RJ,Palmeiras,1,2,A,2.86,3.19,2.77,2.87,3.25,2.80,2.71,3.09,2.64,NaN,NaN,2017-08-03 01:45:00,16610081012943574654,2024-12-18 22:32:46
2268,7683,Brazil,Serie A,2017,Palmeiras,Botafogo RJ,2,0,H,1.72,3.91,5.29,1.82,3.96,5.29,1.73,3.68,4.61,NaN,NaN,2017-11-27 22:00:00,9217996348445472149,2024-12-18 22:32:46
2287,7702,Brazil,Serie A,2018,Botafogo RJ,Palmeiras,1,1,D,3.89,3.37,2.12,3.90,3.38,2.16,3.61,3.23,2.08,NaN,NaN,2018-04-17 00:00:00,18351812539867471458,2024-12-18 22:32:46
2469,7884,Brazil,Serie A,2018,Palmeiras,Botafogo RJ,2,0,H,1.39,4.66,9.92,1.40,4.90,10.02,1.37,4.58,8.68,NaN,NaN,2018-08-23 01:00:00,6091143423004721607,2024-12-18 22:32:46


In [ ]:
# Casa season os times tem desempenhos diferentes, tenho que medir a força a cada season

# No jogo entre Palmeiras vs Botafogo RJ o time visitante venceu o time da casa por 1x3, esse resultado era experado levando em conta o desemenho dos dois times?



In [30]:
df_hist_soccer_matches["home_team"].unique()

array(['Palmeiras', 'Sport Recife', 'Figueirense', 'Botafogo RJ',
       'Corinthians', 'Internacional', 'Ponte Preta', 'Cruzeiro', 'Vasco',
       'Bahia', 'Atletico GO', 'Flamengo RJ', 'Portuguesa', 'Nautico',
       'Sao Paulo', 'Atletico-MG', 'Coritiba', 'Santos', 'Gremio',
       'Fluminense', 'Vitoria', 'Criciuma', 'Athletico-PR', 'Goias',
       'Chapecoense-SC', 'Avai', 'Joinville', 'Santa Cruz', 'America MG',
       'Parana', 'Ceara', 'CSA', 'Fortaleza', 'Bragantino', 'Cuiaba',
       'Juventude'], dtype=object)